In [24]:
import pandas as pd
import re

import platform

os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'

DATA_PATH = PRE_PATH + "data/"
TRAIN_PATH = DATA_PATH + 'train.csv'
DEV_PATH = DATA_PATH + 'dev.csv'
TRAIN_NEW_PATH = DATA_PATH + 'train_new.csv'
DEV_NEW_PATH = DATA_PATH + 'dev_new.csv'

## 오타, 중복 수정(train, dev)

In [25]:

def replace_text(dialogue, replacements):
    for old, new in replacements.items():
        dialogue = re.sub(re.escape(old), new, dialogue)
    return dialogue

def process_file(input_path, output_path, replacements):
    df = pd.read_csv(input_path)
    df['dialogue'] = df['dialogue'].apply(lambda x: replace_text(x, replacements))
    df.to_csv(output_path, index=False)
    print(f"Processed file saved to {output_path}")

if __name__ == "__main__":
    # Define the replacements
    replacements = {
        'ㅋㅋ': '웃겨',
        '배경ㅇ로': '배경으로',
        '제ㅏ': '제가',
        '아직ㅍ알맞는': '아직 알맞는',
        '척했ㄷ거든': '척 했거든',
        '...': '.',
        '!!!': '!',
        '하하하': '하하',
        '아아아아아아': '아아',
        '아아아아아': '아아',
        '아아아아': '아아',
        '아아아': '아아',
        '똑똑똑': '똑똑',
        '키스스스스스': '키스',
        '허허허': '허허',
        '너어어어무': '너무',
        '헤헤헤': '헤헤',
        '으으으으': '으으',
        '제제제': ''
    }

    # Define file paths
    input_files = [TRAIN_PATH, DEV_PATH]
    output_files = [TRAIN_NEW_PATH, DEV_NEW_PATH]

    # Process each file
    for input_path, output_path in zip(input_files, output_files):
        process_file(input_path, output_path, replacements)

Processed file saved to /kkh/data/train_new.csv
Processed file saved to /kkh/data/dev_new.csv


## 오타: 원본 파일에 수정되지 않고 잘 살아 있는지 확인(train)

In [26]:
def find_incomplete_hangul(file_path):
    df = pd.read_csv(file_path)
    if 'dialogue' not in df.columns:
        raise ValueError("'dialogue' 열이 CSV 파일에 없습니다.")
    consonants_and_vowels_pattern = re.compile(r'[ㄱ-ㅎㅏ-ㅣ]+')
    
    results = []
    for _, row in df.iterrows():
        fname = row['fname']
        dialogue = row['dialogue']
        utterances = dialogue.split('\n')  # 각 발화는 줄바꿈으로 구분된다고 가정

        for utterance in utterances:
            incomplete_hangul = re.findall(consonants_and_vowels_pattern, utterance)
            if incomplete_hangul:
                results.append((fname, utterance))
    return results

incomplete_hangul_results = find_incomplete_hangul(TRAIN_PATH)

# 결과 출력
if incomplete_hangul_results:
    for fname, utterance in incomplete_hangul_results:
        print(f"File: {fname}")
        print(f"Utterance: {utterance}")
        print("-" * 40)
else:
    print("No incomplete Hangul characters found.")

File: train_3154
Utterance: #Person1#: 속았어! ㅋㅋ.. 완전 속았어. 
----------------------------------------
File: train_5385
Utterance: #Person2#: 먼저, 이것은 19세기 초 배경ㅇ로 설정된 로맨스 소설이에요.
----------------------------------------
File: train_5429
Utterance: #Person2#: ㅋㅋ
----------------------------------------
File: train_6942
Utterance: #Person2#: 'ㅣ'로 끝나는 이탈리아 이름 같네요. 
----------------------------------------
File: train_7201
Utterance: #Person1#: 편집장이 제ㅏ 다른 잡지에서 편집자로 일했던 경험이 있다는 걸 듣고, 그가 도우미 편집자가 되고 싶냐고 물어봤어요.
----------------------------------------
File: train_9677
Utterance: #Person1#: 이제 그만. 너는 아직ㅍ알맞는 사람을 만나지 못했을 뿐이고, 너는 너무 많이 일하는 것 같아. 너는 어떻게 즐기고 삶을 즐기는 법을 배워야 해.
----------------------------------------
File: train_12181
Utterance: #Person1#: 아무것도 안 했어. 그는 결국 나갔어. 그런데 오늘 또 그를 봤어. 신발 가게 밖에서. 카페 근처에서. 나는 CD 가게에 들어가서 CD를 보는 척했ㄷ거든. 그런데 그도 들어왔어.
----------------------------------------


## 오타: NEW 파일에 잘 적용되었는지 확인(train)

In [27]:
incomplete_hangul_results = find_incomplete_hangul(TRAIN_NEW_PATH)

# 결과 출력
if incomplete_hangul_results:
    for fname, utterance in incomplete_hangul_results:
        print(f"File: {fname}")
        print(f"Utterance: {utterance}")
        print("-" * 40)
else:
    print("No incomplete Hangul characters found.")

File: train_6942
Utterance: #Person2#: 'ㅣ'로 끝나는 이탈리아 이름 같네요. 
----------------------------------------


# 중복: 원본 파일은 수정되지 않고 잘 살아 있는지 확인(train)

In [28]:
def find_repeated_hangul(file_path):
    df = pd.read_csv(file_path)
    if 'dialogue' not in df.columns:
        raise ValueError("'dialogue' 열이 CSV 파일에 없습니다.")
    repeated_hangul_pattern = re.compile(r'([가-힣])\1{2,}')  # 동일 한글 글자가 3번 이상 연속하는 패턴
    
    results = []
    for _, row in df.iterrows():
        fname = row['fname']
        dialogue = row['dialogue']
        
        sentences = dialogue.split('\n')  # 각 문장은 줄바꿈으로 구분된다고 가정
        for sentence in sentences:
            repeated_hangul = re.findall(repeated_hangul_pattern, sentence)
            if repeated_hangul:
                results.append((fname, sentence))
    return results


repeated_hangul_results = find_repeated_hangul(TRAIN_PATH)

# 결과 출력
if repeated_hangul_results:
    for fname, sentence in repeated_hangul_results:
        print(f"File: {fname}")
        print(f"Sentence: {sentence}")
        print("-" * 40)
else:
    print("No repeated Hangul characters found.")

File: train_276
Sentence: #Person1#: 헤이, 그게 뭐예요? [아아아!]
----------------------------------------
File: train_460
Sentence: #Person2#: 아아아아아아
----------------------------------------
File: train_901
Sentence: #Person1#: (고통 속에서 울며) 아아아!
----------------------------------------
File: train_998
Sentence: #Person2#: 하하하, 너 피곤하다고 했잖아. 
----------------------------------------
File: train_1732
Sentence: #Person2#: 아아아아, 아아아아.
----------------------------------------
File: train_1907
Sentence: #Person1#: 아아아악!
----------------------------------------
File: train_2464
Sentence: #Person1#: 그냥 똑똑똑 떨어지는 정도입니다. 그것도 물이 아니죠.
----------------------------------------
File: train_2727
Sentence: #Person1#: 하하하, 아마 그게 너에게 체중 감량에 도움이 될지도 몰라. 
----------------------------------------
File: train_4271
Sentence: #Person2#: 잭, 너의 유머 감각이 그립다. 하하하! 세상에 너처럼 사는 사람은 처음 봤다. 넌 정말 천부적이야.
----------------------------------------
File: train_4401
Sentence: #Person2#: 아아아아악! 침이 고이기 시작하던 참인데!
----------------------------

## 중복: NEW 파일에 잘 적용되었는지 확인(train)

In [29]:
repeated_hangul_results = find_repeated_hangul(TRAIN_NEW_PATH)

# 결과 출력
if repeated_hangul_results:
    for fname, sentence in repeated_hangul_results:
        print(f"File: {fname}")
        print(f"Sentence: {sentence}")
        print("-" * 40)
else:
    print("No repeated Hangul characters found.")

No repeated Hangul characters found.


## 중복: NEW 파일에 잘 적용되었는지 확인(dev)

- dev 원본 확인은 생략하고, 바로 dev_new만 확인한다.

In [30]:
repeated_hangul_results = find_repeated_hangul(DEV_NEW_PATH)

# 결과 출력
if repeated_hangul_results:
    for fname, sentence in repeated_hangul_results:
        print(f"File: {fname}")
        print(f"Sentence: {sentence}")
        print("-" * 40)
else:
    print("No repeated Hangul characters found.")

No repeated Hangul characters found.
